In [1]:
# 필요 라이브러리 import
import os
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

plt.style.use('default')
pd.options.display.max_columns = 100

In [2]:
import os
import pandas as pd
import json

# JSON 파일들이 있는 폴더 경로 설정
folder_path1 = r'D:\데크캠\filtered_labels\training_label'
folder_path2 = r'D:\데크캠\filtered_labels\validation_label'

In [4]:
import os
import pandas as pd
import json

def preference(label_me, output_file):
    # JSON 파일들이 있는 폴더 경로 설정
    folder_path = label_me

    # 데이터프레임을 만들기 위한 리스트 초기화
    records = []

    # 폴더 내 모든 JSON 파일 처리
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)

            # JSON 파일 로드
            with open(file_path, 'r') as f:
                try:
                    data = json.load(f)

                    # 응답자 정보 (user 딕셔너리)
                    r_id = data['user']['R_id']

                    # item 딕셔너리에서 imgName과 Q5 값 추출
                    img_name = data['item']['imgName']
                    q5_value = data['item']['survey']['Q5']

                    # Q5 값에 따라 선호와 비선호 분류
                    if q5_value == 2:
                        preference = img_name  # 스타일 선호
                        records.append({'응답자 ID': r_id, '스타일 선호': preference, '스타일 비선호': None})
                    elif q5_value == 1:
                        preference = img_name  # 스타일 비선호
                        records.append({'응답자 ID': r_id, '스타일 선호': None, '스타일 비선호': preference})

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file_name}: {e}")
                except KeyError as e:
                    print(f"Missing key {e} in file {file_name}")

    # 데이터프레임 생성
    df = pd.DataFrame(records)

    

    # 최종적으로 응답자 ID 기준으로 그룹화하여 스타일 선호와 비선호를 분리
    final_records = []
    for r_id in df['응답자 ID'].unique():
        grouped = df[df['응답자 ID'] == r_id]
        
        for index, row in grouped.iterrows():
            final_records.append({
                '응답자 ID': r_id,
                '스타일 선호': row['스타일 선호'],
                '스타일 비선호': row['스타일 비선호']
            })

    # 최종 데이터프레임 생성
    final_df = pd.DataFrame(final_records)

    # 원하는 파일로 저장 (CSV 파일)
    final_df.to_csv(output_file, encoding='utf-8-sig', index=False)

    print(f"데이터가 {output_file}에 저장되었습니다.")
    
    return final_df  # 최종 데이터프레임 반환


In [5]:
# 함수 사용
df_train = preference(folder_path1,'train_preference.csv')
df_val = preference(folder_path2,'val_preference.csv')

데이터가 train_preference.csv에 저장되었습니다.
데이터가 val_preference.csv에 저장되었습니다.


In [6]:
# csv 불러와서 사용
df_train = pd.read_csv('train_preference.csv')
df_val = pd.read_csv('val_preference.csv')

In [9]:
# 데이터 프레임의 형태 확인하기
df_train.head(20
              )

,응답자 ID,스타일 선호,스타일 비선호
0,52002,NaN,T_00004_90_hiphop_M.jpg
1,52002,NaN,T_03007_10_sportivecasual_M.jpg
2,52002,NaN,T_03118_19_normcore_M.jpg
3,52002,NaN,W_02699_60_mods_M.jpg
4,52002,NaN,W_23983_60_mods_M.jpg
5,52002,W_24111_70_hippie_M.jpg,NaN
6,66699,T_00004_90_hiphop_M.jpg,NaN
7,66699,T_01568_50_ivy_M.jpg,NaN
8,66699,NaN,T_03643_00_metrosexual_M.jpg
9,66699,NaN,T_06009_10_sportivecasual_M.jpg


In [11]:
# 응답자 id별로 선호하는 스타일과 비선호하는 스타일의 NaN 값을 제외한 데이터 개수 확인
train_count = df_train.groupby('응답자 ID').count()
val_count = df_val.groupby('응답자 ID').count()

In [18]:
train_count['train_합계']=train_count['스타일 선호']+train_count['스타일 비선호']
val_count['val_합계']=val_count['스타일 선호']+val_count['스타일 비선호']


In [20]:
train_sum = train_count['train_합계']
val_sum = val_count['val_합계']

                        

In [23]:
# train_sum과 val_sum을 합쳐서 하나의 데이터프레임으로 만들기
df_sum = pd.concat([train_sum, val_sum], axis=1)
df_sum = df_sum.fillna(0).astype(int)
df_sum['합계'] = df_sum['train_합계'] + df_sum['val_합계']
df_sum = df_sum.sort_values(by='합계', ascending=False)

In [25]:
df_sum.head(10)

,train_합계,val_합계,합계
응답자 ID,,,
64747,45,15,60
64561,46,12,58
63405,44,14,58
64346,46,12,58
65139,46,12,58
59704,45,12,57
64223,45,12,57
64310,48,9,57
60173,49,8,57


In [26]:
#df_sum의 합계를 기준으로 상위 100개 응답자 ID 추출하여 리스트로 저장
top_100_ids = df_sum.head(100).index.tolist()


In [29]:
# df_train과 df_valid을 합치는데 '응답자 ID'를 기준으로 합친다., 열의 값은 outer로 설정한다, 응답자 id를 행으로 설정한다
df_all = pd.merge(df_train, df_val, on='응답자 ID', how='outer', suffixes=('_train', '_val')).set_index('응답자 ID')

In [31]:
df_all.head(10)

,스타일 선호_train,스타일 비선호_train,스타일 선호_val,스타일 비선호_val
응답자 ID,,,,
12,W_03412_50_classic_W.jpg,NaN,NaN,W_02651_50_feminine_W.jpg
12,W_03412_50_classic_W.jpg,NaN,W_03412_50_classic_W.jpg,NaN
25,T_01475_10_sportivecasual_M.jpg,NaN,NaN,NaN
25,T_14304_60_mods_M.jpg,NaN,NaN,NaN
25,T_16423_10_sportivecasual_M.jpg,NaN,NaN,NaN
25,NaN,W_12740_00_metrosexual_M.jpg,NaN,NaN
26,T_09447_19_lounge_W.jpg,NaN,NaN,NaN
26,NaN,W_18990_50_feminine_W.jpg,NaN,NaN
27,W_03007_70_hippie_M.jpg,NaN,W_06522_50_ivy_M.jpg,NaN


In [38]:
# df_all에서 top_100_ids에 해당하는 응답자만 추출하고 내림차순으로 정렬
df_top_100 = df_all.loc[top_100_ids].sort_values(by='응답자 ID', ascending=False)

